In [ ]:
import os
import pandas as pd

dir = os.getcwd()
data_dir = os.path.join(dir, 'Data')

department = ['Boyacá', 'Quindío']
coffe_cities = pd.read_excel(os.path.join(data_dir, 'municipios_seleccionados.xlsx'))
stations = pd.read_csv(os.path.join(data_dir, 'Catalogo_Nacional_de_Estaciones_del_IDEAM_20251024.csv'), sep=',')
stations_proy = stations[stations['Departamento'].astype(str).str.strip().isin(department)].copy()

In [37]:
# Unir por Municipio
stations_selected = pd.merge(
    coffe_cities[['municipio']],                # municipios seleccionados
    stations_proy,
    left_on='municipio',
    right_on='Municipio',
    how='inner'                                 # coincidencias exactas
)
stations_selected = stations_selected[['Departamento','Municipio','Codigo','Nombre','Categoria','Tecnologia','LONGITUD','LATITUD','Altitud','Fecha_instalacion','Fecha_suspension']]

In [38]:
stations_selected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Departamento       144 non-null    object 
 1   Municipio          144 non-null    object 
 2   Codigo             144 non-null    int64  
 3   Nombre             144 non-null    object 
 4   Categoria          144 non-null    object 
 5   Tecnologia         144 non-null    object 
 6   LONGITUD           144 non-null    float64
 7   LATITUD            144 non-null    float64
 8   Altitud            144 non-null    object 
 9   Fecha_instalacion  143 non-null    object 
 10  Fecha_suspension   44 non-null     object 
dtypes: float64(2), int64(1), object(8)
memory usage: 12.5+ KB


In [39]:
# Parsear las fechas (maneja strings variados; 'errors="coerce"' pone NaT si no se puede parsear)
for col in ['Fecha_instalacion', 'Fecha_suspension']:
    if col in stations_selected.columns:
        stations_selected[col] = pd.to_datetime(
            stations_selected[col],
            errors='coerce',
            dayfirst=True
        )

# Definir fecha de corte
corte = pd.Timestamp('2010-01-01')

# Construir la máscara:
#    - Mantener registros sin fecha de suspensión (NaT)  -> estación activa o sin dato
#    - Mantener registros con suspensión >= 2010-01-01
mask = stations_selected['Fecha_suspension'].isna() | (stations_selected['Fecha_suspension'] >= corte)

# Aplicar filtro
antes = len(stations_selected)
stations_selected = stations_selected.loc[mask].reset_index(drop=True)
despues = len(stations_selected)

print(f"Estaciones eliminadas por suspensión antes de 2010-01-01: {antes - despues}")
stations_selected.info()


Estaciones eliminadas por suspensión antes de 2010-01-01: 33
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111 entries, 0 to 110
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Departamento       111 non-null    object        
 1   Municipio          111 non-null    object        
 2   Codigo             111 non-null    int64         
 3   Nombre             111 non-null    object        
 4   Categoria          111 non-null    object        
 5   Tecnologia         111 non-null    object        
 6   LONGITUD           111 non-null    float64       
 7   LATITUD            111 non-null    float64       
 8   Altitud            111 non-null    object        
 9   Fecha_instalacion  110 non-null    datetime64[ns]
 10  Fecha_suspension   11 non-null     datetime64[ns]
dtypes: datetime64[ns](2), float64(2), int64(1), object(6)
memory usage: 9.7+ KB


In [40]:
# Conteo de estaciones por Categoria
if 'Categoria' not in stations_selected.columns:
    raise KeyError("La columna 'Categoria' no existe en stations_selected.")

categoria_counts = (
    stations_selected['Categoria']
    .value_counts(dropna=False)
    .rename_axis('Categoria')
    .reset_index(name='Estaciones')
    .sort_values('Estaciones', ascending=False)
)
categoria_counts['Porcentaje'] = (categoria_counts['Estaciones'] / categoria_counts['Estaciones'].sum() * 100).round(2)

print(categoria_counts)

                Categoria  Estaciones  Porcentaje
0           Pluviométrica          40       36.04
1     Climática Principal          27       24.32
2            Limnimétrica          14       12.61
3     Climática Ordinaria          10        9.01
4            Limnigráfica           9        8.11
5  Meteorológica Especial           6        5.41
6           Pluviográfica           4        3.60
7     Sinóptica Principal           1        0.90


In [41]:
# Filtrar estaciones de tipo 'Pluviométrica'
rain_stations = stations_selected[
    stations_selected['Categoria'].str.strip().str.lower() == 'pluviométrica'.lower()
].copy()

# Mostrar un resumen
print(f"\nTotal de estaciones pluviométricas: {rain_stations.shape[0]}")
print(rain_stations[['Departamento', 'Municipio', 'Nombre', 'Tecnologia', 'LONGITUD', 'LATITUD', 'Altitud']])

# Exportar a CSV
rain_stations.to_csv(os.path.join(data_dir, 'rain_stations.csv'), index=False)


Total de estaciones pluviométricas: 40
    Departamento   Municipio                        Nombre    Tecnologia  \
0         Boyacá        Muzo               MUZO [23120050]  Convencional   
3         Boyacá       Pauna          STA RITA  [23120120]  Convencional   
16       Quindío  Buenavista      ESPERANZA LA  [26120590]  Convencional   
20        Boyacá      Caldas            CALDAS  [24010420]  Convencional   
26        Boyacá   Somondoco          SOMONDOCO [35070100]  Convencional   
27        Boyacá   Somondoco            MARTOTA [35070320]  Convencional   
31       Quindío     Armenia       GOBERNACION  [26120320]  Convencional   
32       Quindío     Armenia        PRADERA LA  [26120500]  Convencional   
33       Quindío     Armenia           TUCUMAN  [26120520]  Convencional   
37       Quindío  Buenavista      ESPERANZA LA  [26120590]  Convencional   
41       Quindío     Córdoba        CORDOBA LA  [26120350]  Convencional   
42       Quindío     Córdoba            MONACO  